# SQL practice playground

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/rsannareddy/playground/blob/main/sql_playground.ipynb)


In [ ]:
import sqlite3
from textwrap import dedent

DB_NAME = "practice_sql.db"

In [ ]:
def setup_database(conn):
    """
    Create sample tables and seed them with data.
    This function is idempotent: you can run it multiple times safely.
    """
    cur = conn.cursor()

    # Enable foreign keys (off by default in SQLite)
    cur.execute("PRAGMA foreign_keys = ON;")

    # Create tables
    cur.executescript(
        dedent(
            """
            CREATE TABLE IF NOT EXISTS customers (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                city TEXT,
                join_date DATE
            );

            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                category TEXT,
                price REAL NOT NULL
            );

            CREATE TABLE IF NOT EXISTS orders (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer_id INTEGER NOT NULL,
                order_date DATE NOT NULL,
                status TEXT NOT NULL,
                FOREIGN KEY (customer_id) REFERENCES customers(id)
            );

            CREATE TABLE IF NOT EXISTS order_items (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                order_id INTEGER NOT NULL,
                product_id INTEGER NOT NULL,
                quantity INTEGER NOT NULL,
                FOREIGN KEY (order_id) REFERENCES orders(id),
                FOREIGN KEY (product_id) REFERENCES products(id)
            );
            """
        )
    )

    # Insert sample data only if tables are empty
    cur.execute("SELECT COUNT(*) FROM customers;")
    (customer_count,) = cur.fetchone()

    if customer_count == 0:
        # Customers
        cur.executemany(
            "INSERT INTO customers (name, city, join_date) VALUES (?, ?, ?);",
            [
                ("Arjun", "Hyderabad", "2023-01-15"),
                ("Bhima", "Bengaluru", "2023-02-10"),
                ("Karna", "Chennai", "2023-02-20"),
                ("Draupadi", "Hyderabad", "2023-03-05"),
            ],
        )

        # Products
        cur.executemany(
            "INSERT INTO products (name, category, price) VALUES (?, ?, ?);",
            [
                ("Laptop", "Electronics", 70000),
                ("Mouse", "Electronics", 800),
                ("Chair", "Furniture", 4500),
                ("Table", "Furniture", 6500),
                ("Headphones", "Electronics", 3500),
            ],
        )

        # Orders
        cur.executemany(
            "INSERT INTO orders (customer_id, order_date, status) VALUES (?, ?, ?);",
            [
                (1, "2023-03-01", "Completed"),
                (1, "2023-03-15", "Completed"),
                (2, "2023-03-10", "Pending"),
                (3, "2023-03-12", "Cancelled"),
                (4, "2023-03-20", "Completed"),
            ],
        )

        # Order items
        cur.executemany(
            "INSERT INTO order_items (order_id, product_id, quantity) VALUES (?, ?, ?);",
            [
                (1, 1, 1),  # Arjun buys 1 Laptop
                (1, 2, 2),  # Arjun buys 2 Mice
                (2, 3, 1),  # Arjun buys 1 Chair
                (3, 5, 3),  # Bhima buys 3 Headphones (order pending)
                (4, 2, 1),  # Karna (cancelled order)
                (5, 1, 1),  # Draupadi buys 1 Laptop
                (5, 4, 1),  # Draupadi buys 1 Table
            ],
        )

        print("Sample data inserted.")
    else:
        print("Database already has data, skipping seeding.")

    conn.commit()

In [ ]:
def print_results(cursor):
    """
    Nicely print the results of a SELECT query.
    """
    rows = cursor.fetchall()
    if not rows:
        print("(no rows returned)")
        return

    # Use column names from cursor.description
    col_names = [desc[0] for desc in cursor.description]

    # Compute column widths
    str_rows = [[str(value) for value in row] for row in rows]
    col_widths = [
        max(len(col_names[i]), max(len(r[i]) for r in str_rows))
        for i in range(len(col_names))
    ]

    # Print header
    header = " | ".join(
        col_names[i].ljust(col_widths[i]) for i in range(len(col_names))
    )
    sep = "-+-".join("-" * col_widths[i] for i in range(len(col_widths)))
    print(header)
    print(sep)

    # Print rows
    for r in str_rows:
        line = " | ".join(r[i].ljust(col_widths[i]) for i in range(len(r)))
        print(line)

In [ ]:
def main():
    # Connect to SQLite database file (will create if it doesn't exist)
    conn = sqlite3.connect(DB_NAME)

    # Optional: access results by column name if you want
    conn.row_factory = sqlite3.Row

    setup_database(conn)

    cur = conn.cursor()

    print("\nSQL practice REPL")
    print(f"Connected to SQLite database: {DB_NAME}")
    print("Type any SQL query.")
    print("Type 'tables;' to list tables.")
    print("Type 'exit', 'quit', or 'q' to leave.\n")

    while True:
        user_input = input("SQL> ").strip()

        # Simple helper commands
        if user_input.lower() in {"exit", "quit", "q"}:
            break
        if user_input.lower() == "tables;":
            cur.execute(
                "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
            )
            print_results(cur)
            continue
        if not user_input:
            continue

        try:
            cur.execute(user_input)

            # Decide if it's a SELECT query
            if user_input.lstrip().lower().startswith("select"):
                print_results(cur)
            else:
                conn.commit()
                print(f"(Query OK, {cur.rowcount} row(s) affected)")
        except Exception as e:
            print(f"Error: {e}")

    conn.close()
    print("Connection closed. Bye!")

In [ ]:
main()

In [ ]:
elect * from orders inner join orders on orders.customer_id = customers.id;
